---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [1]:
import pandas as pd
import numpy as np

In [2]:
training = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
testing = pd.read_csv('test.csv')
adr2latlons = pd.read_csv('latlons.csv')
id2adr = pd.read_csv('addresses.csv')

c:\Miniconda3\envs\tf-gpu\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
X_train = training[testing.columns]
y_train = training['compliance']

In [4]:
X_train = X_train[~y_train.isnull()]
y_train = y_train[~y_train.isnull()]

In [5]:
X_train = pd.merge(pd.merge(X_train, id2adr, on ='ticket_id', how='left'), 
                   adr2latlons, on='address', how='left').drop('address', axis =1)

In [6]:
X_test = pd.merge(pd.merge(testing, id2adr, on ='ticket_id', how='left'), 
                   adr2latlons, on='address', how='left').drop('address', axis =1)

In [7]:
X_train.fillna(method='pad', inplace=True)
X_test.fillna(method='pad', inplace=True)

In [8]:
X_train.drop('violation_zip_code', axis=1, inplace=True)
X_test.drop('violation_zip_code', axis=1, inplace=True)

In [9]:
X_train.drop('grafitti_status', axis=1, inplace=True)
X_test.drop('grafitti_status', axis=1, inplace=True)

In [10]:
X_train.drop('violation_street_number', axis=1, inplace=True)
X_test.drop('violation_street_number', axis=1, inplace=True)

In [11]:
X_train.drop('mailing_address_str_number', axis=1, inplace=True)
X_test.drop('mailing_address_str_number', axis=1, inplace=True)

In [12]:
X_train.drop('non_us_str_code', axis=1, inplace=True)
X_test.drop('non_us_str_code', axis=1, inplace=True)

In [13]:
X_train.drop('country', axis=1, inplace=True)
X_test.drop('country', axis=1, inplace=True)

In [14]:
X_train.drop('violation_description', axis=1, inplace=True)
X_test.drop('violation_description', axis=1, inplace=True)

In [15]:
X_train.drop('ticket_id', axis=1, inplace=True)
X_test.drop('ticket_id', axis=1, inplace=True)

In [16]:
X_train.drop('inspector_name', axis=1, inplace=True)
X_test.drop('inspector_name', axis=1, inplace=True)

In [17]:
X_train.drop('zip_code', axis=1, inplace=True)
X_test.drop('zip_code', axis=1, inplace=True)

In [18]:
X_train.drop('city', axis=1, inplace=True)
X_test.drop('city', axis=1, inplace=True)
X_train.drop('state', axis=1, inplace=True)
X_test.drop('state', axis=1, inplace=True)

In [19]:
X_train.drop('violation_street_name', axis=1, inplace=True)
X_test.drop('violation_street_name', axis=1, inplace=True)
X_train.drop('mailing_address_str_name', axis=1, inplace=True)
X_test.drop('mailing_address_str_name', axis=1, inplace=True)

In [20]:
X_train.drop('violator_name', axis=1, inplace=True)
X_test.drop('violator_name', axis=1, inplace=True)

<br>
<br>
** coursera autograder server fail to process datetime using pandas, maybe version problem**

In [ ]:
#X_train['date_gap'] = X_train['hearing_date'].astype('datetime64[D]') - X_train['ticket_issued_date'].astype('datetime64[D]')
#X_test['date_gap'] = X_test['hearing_date'].astype('datetime64[D]') - X_test['ticket_issued_date'].astype('datetime64[D]')
X_train.drop('hearing_date', axis=1, inplace=True)
X_test.drop('hearing_date', axis=1, inplace=True)
X_train.drop('ticket_issued_date', axis=1, inplace=True)
X_test.drop('ticket_issued_date', axis=1, inplace=True)

In [ ]:
#X_train['date_gap'] = X_train['date_gap'].dt.total_seconds() / (24 * 60 * 60)
#X_test['date_gap'] = X_test['date_gap'].dt.total_seconds() / (24 * 60 * 60)

In [ ]:
X_train.drop(['fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'clean_up_cost'], axis=1, inplace=True)
X_test.drop(['fine_amount','admin_fee','state_fee','late_fee', 'clean_up_cost'], axis=1, inplace=True)

In [ ]:
X_train['total_amount'] = X_train['judgment_amount'] - X_train['discount_amount']
X_test['total_amount'] = X_test['judgment_amount'] - X_test['discount_amount']
X_train.drop(['judgment_amount', 'discount_amount'], axis=1, inplace=True)
X_test.drop(['judgment_amount', 'discount_amount'], axis=1, inplace=True)

In [ ]:
agency_ix = (X_test.agency_name.values[:, None] == X_train.agency_name.unique()[None, :]).sum(axis=1).astype(bool)  == False
violation_code_ix = (X_test.violation_code.values[:, None] == X_train.violation_code.unique()[None, :]).sum(axis=1).astype(bool)  == False
disposition_ix = (X_test.disposition.values[:, None] == X_train.disposition.unique()[None, :]).sum(axis=1).astype(bool)  == False

In [ ]:
X_test.loc[agency_ix, 'agency_name'] = 'unkown'
X_test.loc[violation_code_ix, 'violation_code'] = 'unkown'
X_test.loc[disposition_ix, 'disposition'] = 'unkown'

In [ ]:
from sklearn import preprocessing
le_agency = preprocessing.LabelEncoder().fit(X_train.agency_name.unique().tolist() + ['unkown'])
le_violation_code = preprocessing.LabelEncoder().fit(X_train.violation_code.unique().tolist() + ['unkown'])
le_disposition = preprocessing.LabelEncoder().fit(X_train.disposition.unique().tolist() + ['unkown'])

In [ ]:
X_train['agency_name'] = le_agency.transform(X_train['agency_name'])
X_train['violation_code'] = le_violation_code.transform(X_train['violation_code'])
X_train['disposition'] = le_disposition.transform(X_train['disposition'])

X_test['agency_name'] = le_agency.transform(X_test['agency_name'])
X_test['violation_code'] = le_violation_code.transform(X_test['violation_code'])
X_test['disposition'] = le_disposition.transform(X_test['disposition'])

In [ ]:
enc_agency = preprocessing.OneHotEncoder().fit(np.arange(le_agency.classes_.size)[:, None])
enc_violation_code = preprocessing.OneHotEncoder().fit(np.arange(le_violation_code.classes_.size)[:, None])
enc_disposition = preprocessing.OneHotEncoder().fit(np.arange(le_disposition.classes_.size)[:, None])

In [ ]:
X_train_agency = enc_agency.transform(X_train.agency_name[:, None]).toarray()
X_train_violation_code = enc_violation_code.transform(X_train.violation_code[:, None]).toarray()
X_train_disposition = enc_disposition.transform(X_train.disposition[:, None]).toarray()

X_test_agency = enc_agency.transform(X_test.agency_name[:, None]).toarray()
X_test_violation_code = enc_violation_code.transform(X_test.violation_code[:, None]).toarray()
X_test_disposition = enc_disposition.transform(X_test.disposition[:, None]).toarray()

In [ ]:
X_train = np.hstack([X_train.loc[:, X_train.columns[-4:]], X_train_agency, X_train_violation_code, X_train_disposition])
X_test = np.hstack([X_test.loc[:, X_test.columns[-4:]], X_test_agency, X_test_violation_code, X_test_disposition])

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

```python
n_estimators
grid_values = {'n_estimators': [50, 100, 200, 400, 800], 'max_depth':[3, 4, 5], 'learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]  }
clf = GradientBoostingClassifier()
grid_clf_auc = GridSearchCV(clf, cv=3, n_jobs=-1, param_grid = grid_values, scoring = 'roc_auc')
grid_clf_auc.fit(X_train, y_train)
y_decision_fn_scores_auc = grid_clf_auc.decision_function(X_test) 

#print('Test set AUC: ', roc_auc_score(y_test, y_decision_fn_scores_auc))
print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
print('Grid best score (AUC): ', grid_clf_auc.best_score_)
```

```python
clf = GradientBoostingClassifier(n_estimators=200, learning_rate=0.01).fit(X_train, y_train)

y_decision_fn = clf.decision_function(X_train) 
roc_auc_score(y_train, y_decision_fn)
```

In [ ]:
import pandas as pd
import numpy as np

def blight_model():
    
    # Your code here
    clf = GradientBoostingClassifier(n_estimators=20, learning_rate=0.1).fit(X_train, y_train)
    y_proba = clf.predict_proba(X_test) 
    result = pd.Series(y_proba[:,1], index=testing.ticket_id, name='compliance')
    return result # Your answer here

### check

```python
bm = blight_model()
res = 'Data type Test: '
res += ['Failed: type(bm) should Series\n','Passed\n'][type(bm)==pd.Series]
res += 'Data shape Test: '
res += ['Failed: len(bm) should be 61001\n','Passed\n'][len(bm)==61001]
res += 'Data Values Test: '
res += ['Failed: all values should be in [0.,1.]\n','Passed\n'][all((bm<=1.) & (bm>=0.))]
res += 'Data Values type Test: '
res += ['Failed: bm.dtype should be float\n','Passed\n'][str(bm.dtype).count('float')>0]
res += 'Index type Test: '
res += ['Failed: type(bm.index) should be Int64Index\n','Passed\n'][type(bm.index)==pd.Int64Index]
res += 'Index values type Test: '
res += ['Failed: type(bm.index[0]) should be int64\n','Passed\n'][str(type(bm.index[0])).count("int64")>0]

res += 'Output index shape test:'
res += ['Failed, bm.index.shape should be (61001,)\n','Passed\n'][bm.index.shape==(61001,)]

res += 'Output index test: '
if bm.index.shape==(61001,):
    res +=['Failed\n','Passed\n'][all(pd.read_csv('test.csv',usecols=[0],index_col=0).sort_index().index.values==bm.sort_index().index.values)]
else:
    res+='Failed'
print(res)
```